In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import plotly.graph_objects as go
import gradio as gr

In [5]:
# XOR dataset
X = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
y = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)


In [6]:
# Define the Neural Network Model
class XORNet(nn.Module):
    def __init__(self):
        super(XORNet, self).__init__()
        self.hidden = nn.Linear(2, 2)
        self.output = nn.Linear(2, 1)

    def forward(self, x):
        x = torch.sigmoid(self.hidden(x))
        x = torch.sigmoid(self.output(x))
        return x

In [8]:
# Instantiate the model
model = XORNet()

# Define the Loss Function and Optimizer
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [10]:
# Training loop
epochs = 10000
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 1000 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


Epoch [1000/10000], Loss: 0.6932
Epoch [2000/10000], Loss: 0.6929
Epoch [3000/10000], Loss: 0.6919
Epoch [4000/10000], Loss: 0.6812
Epoch [5000/10000], Loss: 0.6113
Epoch [6000/10000], Loss: 0.4738
Epoch [7000/10000], Loss: 0.1969
Epoch [8000/10000], Loss: 0.0802
Epoch [9000/10000], Loss: 0.0469
Epoch [10000/10000], Loss: 0.0325


In [11]:
# Evaluate the model
with torch.no_grad():
    predicted = model(X).round()
    print(f'Predicted outputs:\n{predicted}\n')
    print(f'Actual outputs:\n{y}\n')
    accuracy = (predicted == y).sum().item() / y.size(0)
    print(f'Accuracy: {accuracy * 100}%')

Predicted outputs:
tensor([[0.],
        [1.],
        [1.],
        [0.]])

Actual outputs:
tensor([[0.],
        [1.],
        [1.],
        [0.]])

Accuracy: 100.0%


In [12]:
# Function to visualize the decision boundary
def visualize_decision_boundary(model, X, y):
    xx, yy = np.meshgrid(np.linspace(0, 1, 100), np.linspace(0, 1, 100))
    grid = np.c_[xx.ravel(), yy.ravel()]
    grid_tensor = torch.tensor(grid, dtype=torch.float32)

    with torch.no_grad():
        Z = model(grid_tensor).numpy().reshape(xx.shape)

    fig = go.Figure(data=go.Contour(z=Z, x0=0, y0=0, dx=0.01, dy=0.01, showscale=False))
    fig.add_trace(go.Scatter(x=X[:, 0], y=X[:, 1], mode='markers', marker=dict(color=y.flatten(), size=10, line=dict(color='black', width=1))))
    fig.update_layout(title='XOR Decision Boundary', xaxis_title='X1', yaxis_title='X2')

    return fig

In [13]:
# Function to predict using the model
def predict(x1, x2):
    with torch.no_grad():
        input_tensor = torch.tensor([[x1, x2]], dtype=torch.float32)
        output = model(input_tensor).item()
    return round(output)

# Visualize the decision boundary
fig = visualize_decision_boundary(model, X, y.numpy())
fig.show()

# Launch Gradio interface
gr.Interface(fn=predict, inputs=["number", "number"], outputs="number").launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b5aa8441da043c664e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
